# PROJECT: Data Analysis of Yandex.Music - Comparing Users in Two Cities

**Research Goal** To test three hypotheses:

Hypothesis 1: User activity depends on the day of the week, manifesting differently in Moscow and St. Petersburg.

    H0: User activity is not dependent on the day of the week, and there are no differences in activity between Moscow and St. Petersburg.

    H1: User activity is dependent on the day of the week, and there are differences in activity between Moscow and St. Petersburg.

Hypothesis 2: Different music genres dominate on Monday mornings and Friday evenings depending on the city (Moscow/St. Petersburg).

    H0: There are no differences in the dominant music genres between Moscow and St. Petersburg on Monday mornings and Friday evenings.

    H1: There are differences in the dominant music genres between Moscow and St. Petersburg on Monday mornings and Friday evenings.

Hypothesis 3: Moscow and St. Petersburg prefer different music genres: Moscow prefers pop music, St. Petersburg prefers Russian rap.

    H0: There are no differences in music preferences between Moscow and St. Petersburg.

    H1: Moscow prefers pop music, and St. Petersburg prefers Russian rap.

**ASSUMPTIONS**

In practice, research often includes statistical hypothesis testing. While we did not use such tests in this project, it's important to note that these tests could provide valuable insights into the reliability of our findings based on the available data.

It's crucial to acknowledge that data from a single service may not always be representative of the entire population of a city. Our analysis relies solely on information from users of one specific service.

## Setup

### Imports

In [27]:
import pandas as pd

### Functions

In [28]:
# Function number_tracks() with two parameters: day, city.
def number_tracks(day, city):
    '''
    Function calculates the number of tracks listened to for a given day and city
    '''
    # The track_list variable stores the rows of the df table for which
    # the value in the 'day' column equals the day parameter and at the same time the value
    # in the 'city' column is equal to the city parameter (use sequential filtering
    # using boolean indexing or complex boolean expressions in a single line if you are already familiar with them).
    track_list = df[(df['day'] == day) & (df['city'] == city)]

    # The track_list_count variable stores the number of values in the 'user_id' column,
    # calculated by the count() method for the track_list table.
    track_list_count = track_list['user_id'].count()

    # The function returns a number - the value of track_list_count.
    return track_list_count

# Declaration of the genre_weekday() function with parameters df, day, time1, time2,
# which returns information about the most popular genres on the specified day in
# the given time:
# 1) the genre_df variable stores the rows of the passed dataframe df for which
#    at the same time:
#    - the value in the day column equals the value of the day argument
#    - the value in the time column is greater than the value of the time1 argument
#    - the value in the time column is less than the value of the time2 argument
#    Use sequential filtering using boolean indexing.
# 2) group the genre_df dataframe by the genre column, take one of its
#    columns and count the number of records for each
#    genres present, write the resulting Series to a variable
#    genre_df_grouped
# 3) sort genre_df_grouped by descending frequency and save
#    to variable genre_df_sorted
# 4) return a Series from the first 10 values of genre_df_sorted, these will be the top 10
#    popular genres (on the specified day, at the specified time)
def genre_weekday(df, day, time1, time2):
    # sequential filtering
    # leave only those rows of df in genre_df for which the day is equal to day
    genre_df = df[df['day'] == day] 
    # leave only those rows of genre_df in genre_df for which the time is less than time2
    genre_df = genre_df[genre_df['time'] < time2] 
    # leave only those rows of genre_df in genre_df for which the time is greater than time1
    genre_df = genre_df[genre_df['time'] > time1] 
    # group the filtered dataframe by the column with genre names, take the genre column and count the number of rows for each genre using the count() method
    genre_df_grouped = genre_df.groupby('genre')['genre'].count()
    # sort the result in descending order (so that the most popular genres are at the beginning of the Series)
    genre_df_sorted = genre_df_grouped.sort_values(ascending=False) 
    # return a Series with the 10 most popular genres in the specified time interval of a given day
    return genre_df_sorted[:10]

## EDA

In [29]:
# Читаем датасет
df = pd.read_csv('yandex_music_project.csv')

In [30]:
# Посмотрим содержимое первых строк
print(df.head(10))

     userID                        Track            artist   genre  \
0  FFB692EC            Kamigata To Boots  The Mass Missile    rock   
1  55204538  Delayed Because of Accident  Andreas Rönnberg    rock   
2    20EC38            Funiculì funiculà       Mario Lanza     pop   
3  A3DD03C9        Dragons in the Sunset        Fire + Ice    folk   
4  E2DC1FAE                  Soul People        Space Echo   dance   
5  842029A1                    Преданная         IMPERVTOR  rusrap   
6  4CB90AA5                         True      Roman Messer   dance   
7  F03E1C1F             Feeling This Way   Polina Griffith   dance   
8  8FA1D3BE     И вновь продолжается бой               NaN  ruspop   
9  E772D5C0                    Pessimist               NaN   dance   

             City        time        Day  
0  Saint-Petersburg  20:28:33  Wednesday  
1            Moscow  14:07:09     Friday  
2  Saint-Petersburg  20:58:07  Wednesday  
3  Saint-Petersburg  08:37:09     Monday  
4            M

**Промежуточный вывод**

В каждом объекте содержатся данные о прослушанном треке. Часть признаков описывают трек, другие хранят информацию о слушателе(города, дата, время прослушивания)

Имеющихся признаков достаточно, для рабты по выдвинутым гипотезам.

In [31]:
# Изучим сводную характеристику датасета
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


**Промежуточный вывод**

В датасете всего семь признаков. У всех признаков тип  `object`.
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

Видим что есть пропуски в данных так как количество значений в признаках разные. 
Так же видим нарушения стилистики в названиях признаков:
- Разный регистр букв, все должно быть оформленно прописными буквами.
- Наличие пробелов в названиях не допустимо.



### Стиль заголовков

Исправления стиля названий признаков

In [32]:
# Еще раз посмотрим на названия признаков
print(df.columns)

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')


Приведем названия в соответствие с PEP-8:
* несколько слов в названии записываются в «змеином_регистре»
* все символы строчные
* отсутствие пробелов.

In [33]:
# Переименуем признаки так
df = df.rename(columns={'  userID': 'user_id', 'Track': 'track', '  City  ': 'city', 'Day' : 'day' })

In [34]:
# Проверка
print(df.columns)

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')


### Пропуски значений

In [35]:
# Посмотрим сколько пропусков в каждом признаке
print(df.isna().sum())

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64


**Промежуточные выводы**

- Пропуски в `track` и `artist` не важны для нашей задачи, поэтому заменим их явными обозначениями.
- Пропуски в `genre` могут помешать сравнить музыкальные вкусы в Москве и Санкт-Петербурге. Поэтому заполним и эти пропуски явными обозначениями;
  
Обязательно необходимо будет провести оценку, насколько наши заполнения повлдияют на расчёты.

In [36]:
# Заменим пропущенные значения в столбцах track, artist и genre на строку 'unknown'
columns_to_replace = ['track', 'artist', 'genre']
for column in columns_to_replace:
    df[column] = df[column].fillna('unknown')     

In [37]:
# Проверка на отсутствие пропусков
print(df.isna().sum()) 

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64


### Дубликаты

#### Явные дубликаты

In [38]:
# Посмотрим на количество явных дубликаты
print(df.duplicated().sum()) 

3826


**Промежуточные выводы**

3k дубликатов из 65k Объектов это ничтожно малая цифра, удалим.

In [39]:
# Удалим все явные дубликаты
df = df.drop_duplicates()

In [40]:
# Проверка на отсутствие дублей
print(df.duplicated().sum()) 

0


#### Неявные дубликаты

Теперь избавьтесь от неявных дубликатов в колонке `genre`. Например, название одного и того же жанра может быть записано немного по-разному. Такие ошибки тоже повлияют на результат исследования.

**Задание 15**

Выведите на экран список уникальных названий жанров, отсортированный в алфавитном порядке. Для этого:
* извлеките нужный столбец датафрейма;
* примените к нему метод сортировки;
* для отсортированного столбца вызовите метод, который вернёт уникальные значения из столбца.

In [41]:
genres = df['genre']
genres = genres.sort_values()
print(list(genres.unique()))

['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans', 'alternative', 'alternativepunk', 'ambient', 'americana', 'animated', 'anime', 'arabesk', 'arabic', 'arena', 'argentinetango', 'art', 'audiobook', 'author', 'avantgarde', 'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass', 'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks', 'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean', 'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill', 'chinese', 'choral', 'christian', 'christmas', 'classical', 'classicmetal', 'club', 'colombian', 'comedy', 'conjazz', 'contemporary', 'country', 'cuban', 'dance', 'dancehall', 'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr', 'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo', 'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic', 'electropop', 'emo', 'entehno', 'epicmetal', 'estrada', 'ethnic', 'eurofolk', 'european', 'experimental', 'extrememetal', 'fado', 'fairyt

**Промежуточные выводы**

Видим следующие неявные дубликаты:
* *hip*,
* *hop*,
* *hip-hop*.

Заменим все вместо `hip`, `hop` и `hip-hop` на значение `hiphop`:

In [42]:
# список дубликатов
duplicate_names = ['hip', 'hop', 'hip-hop']
# правильное наименование
with_name = 'hiphop'

# замена
df['genre'] = df['genre'].replace(duplicate_names, with_name)

In [43]:
# Проверка
print(list(df['genre'].sort_values().unique()))

['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans', 'alternative', 'alternativepunk', 'ambient', 'americana', 'animated', 'anime', 'arabesk', 'arabic', 'arena', 'argentinetango', 'art', 'audiobook', 'author', 'avantgarde', 'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass', 'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks', 'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean', 'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill', 'chinese', 'choral', 'christian', 'christmas', 'classical', 'classicmetal', 'club', 'colombian', 'comedy', 'conjazz', 'contemporary', 'country', 'cuban', 'dance', 'dancehall', 'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr', 'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo', 'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic', 'electropop', 'emo', 'entehno', 'epicmetal', 'estrada', 'ethnic', 'eurofolk', 'european', 'experimental', 'extrememetal', 'fado', 'fairyt

**Выводы**

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

Мы исправили заголовки, чтобы упростить работу с таблицей. Устранили явные и неявные дубликаты, без них исследование станет более точным.

Пропуски заменили на `'unknown'`.

## Проверка гипотез

### Гипотеза 1. Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. 
Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. 

Для этого:

* Разделим пользователей Москвы и Санкт-Петербурга.
* Сравним, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.


In [44]:
# Сгруппируем данные по городу и посчитаем прослушивания в каждой группе, для оценки активности пользователей.
print(df.groupby('city')['time'].count())

city
Moscow              42741
Saint-Petersburg    18512
Name: time, dtype: int64


**Промежуточный вывод**

В Москве прослушиваний больше, чем в Петербурге. Скорее всего это объясняется тем, что в Москве пользователей в разы больше.


In [45]:
# Cгруппируем данные по дню недели и расчитаем прослушивания в понедельник, среду и пятницу.

print(df.groupby('day')['time'].count())

day
Friday       21840
Monday       21354
Wednesday    18059
Name: time, dtype: int64


**Промежуточный вывод**

В среднем пользователи в совокупоности обоих городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

Посмотрим как респределено количество прослушиваний между пользователями различных городов по дням

In [46]:
number_tracks('Monday', 'Moscow')

15740

In [47]:
number_tracks('Monday', 'Saint-Petersburg')

5614

In [48]:
number_tracks('Wednesday', 'Moscow')

11056

In [49]:
number_tracks('Wednesday', 'Saint-Petersburg')

7003

In [50]:
number_tracks('Friday', 'Moscow')

15945

In [51]:
number_tracks('Friday', 'Saint-Petersburg')

5895

In [52]:
# данные — результаты, которые получили с помощью number_tracks
data = [['Moscow',15740,11056,15945],
        ['Saint-Petersburg',5614,7003,5614]]
# названия колонок
columns = ['city','monday','wednesday','friday']
# таблица
table = pd.DataFrame(data = data, columns = columns)
print(table)

               city  monday  wednesday  friday
0            Moscow   15740      11056   15945
1  Saint-Petersburg    5614       7003    5614


**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Гипотеза 2. Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

In [53]:
# получим таблицу moscow_general из тех строк таблицы df,
# для которых значение в столбце 'city' равно 'Moscow'
moscow_general=df[df['city'] == 'Moscow']

In [54]:
# получим таблицу spb_general из тех строк таблицы df,
# для которых значение в столбце 'city' равно 'Saint-Petersburg'
spb_general=df[df['city']=='Saint-Petersburg']

In [55]:
# Вызовем функцию genre_weekday()для утра понедельника в Москве (вместо df — таблица moscow_general)
# объекты, хранящие время, являются строками и сравниваются как строки
# пример вызова: genre_weekday(moscow_general, 'Monday', '07:00', '11:00')
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [56]:
# вызов функции для утра понедельника в Петербурге (вместо df — таблица spb_general)
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [57]:
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [58]:
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное различие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Гипотеза 3. Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

In [59]:
# одной строкой: группировка таблицы moscow_general по столбцу 'genre',
# подсчёт числа значений 'genre' в этой группировке методом count(),
# сортировка получившегося Series в порядке убывания и сохранение в moscow_genres
moscow_genres=moscow_general.groupby('genre')['genre'].count().sort_values(ascending=False)



In [60]:
# Посмотрим на первые десять строк moscow_genres
print(moscow_genres.head(10))

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64


In [61]:
# одной строкой: группировка таблицы spb_general по столбцу 'genre',
# подсчёт числа значений 'genre' в этой группировке методом count(),
# сортировка получившегося Series в порядке убывания и сохранение в spb_genres
spb_genres=spb_general.groupby('genre')['genre'].count().sort_values(ascending=False)


In [62]:
# Посмотрим на первые десять строк # Посмотрим на первые десять строк moscow_genres
print(spb_genres.head(10))

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64


**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге.


## Итоги исследования

Мы проверили три гипотезы и установили:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге.

- Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
- в Москве слушают музыку жанра “world”,
- в Петербурге — джаз и классику.

- Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего, чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

- Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.